In [1]:
import os
from pathlib import Path
import shutil

import vak

In [2]:
BFSongRepo = Path('/home/bart/Documents/data/birdsong/BFSongRepository/')

In [3]:
all_notmats = list(BFSongRepo.glob('*/*/*.not.mat'))
bird_date_dirs = set([notmat.parents[0] for notmat in all_notmats])

In [28]:
for bird_date_dir in bird_date_dirs:
    has_notmat = bird_date_dir.joinpath('has_notmat')
    has_notmat.mkdir(exist_ok=True)
    notmats_this_date_dir = sorted(list(bird_date_dir.glob('*.not.mat')))
    for notmat in notmats_this_date_dir:
        shutil.copy(notmat, dst=has_notmat)
        cbin = notmat.parent.joinpath(
            Path(notmat.stem).stem
        )
        shutil.copy(cbin, dst=has_notmat)  # cbin_file, stem.stem removes .not.mat
        rec = notmat.parent.joinpath(
            Path(Path(notmat.stem).stem).stem + '.rec'
        )
        shutil.copy(rec, dst=has_notmat)
        tmp = notmat.parent.joinpath(
            Path(Path(notmat.stem).stem).stem + '.tmp'
        )
        shutil.copy(tmp, dst=has_notmat)

In [4]:
BIRDS = ['bl26lb16', 'gy6or6', 'or60yw70', 'gr41rd51']

In [5]:
dirs_to_predict = {}
for bird in BIRDS:
    dirs_to_predict[bird] = [
        bird_date_dir for bird_date_dir in bird_date_dirs
        if bird in str(bird_date_dir)
    ]

In [8]:
vak.dataset.prep(
    data_dir,
    audio_format='cbin'
    spect_params=spect_params
)

Signature:
vak.dataset.prep(
    data_dir,
    annot_format=None,
    labelset=None,
    output_dir=None,
    save_vds=False,
    vds_fname=None,
    return_vds=True,
    return_path=True,
    load_spects=True,
    annot_file=None,
    audio_format=None,
    spect_format=None,
    spect_params=None,
    spect_output_dir=None,
)
Docstring:
prepare a VocalizationDataset from a directory of audio or spectrogram files
containing vocalizations, and (optionally) annotation for those files

Parameters
----------
data_dir : str
    path to directory with audio or spectrogram files from which to make dataset
annot_format : str
    format of annotations. Any format that can be used with the
    crowsetta library is valid. Default is None.
labelset : set, list
    of str or int, set of labels for vocalizations. Default is None.
    If not None, then files will be skipped where the 'labels' array in the
    corresponding annotation contains labels that are not found in labelset
output_dir : str
  

In [7]:
vak.cli.predict(
    checkpoint_path,
    networks,
    labels_mapping_path,
    spect_params,
    dir_to_predict=None,
    mat_spect_files_path=None,
    spect_scaler_path=None,
)

Signature:
vak.cli.predict(
    checkpoint_path,
    networks,
    labels_mapping_path,
    spect_params,
    dir_to_predict=None,
    mat_spect_files_path=None,
    spect_scaler_path=None,
)
Docstring:
make predictions with one trained model

Parameters
----------
checkpoint_path : str
    path to directory with saved model
networks : dict
    where each key is the name of a neural network and the corresponding
    value is the configuration for that network (in a namedtuple or a dict)
labels_mapping_path : str
    path to file that contains labels mapping, to convert output from consecutive
    digits back to labels used for audio segments (e.g. birdsong syllables)
spect_params : dict
    Dictionary of parameters for creating spectrograms.
dir_to_predict : str
    path to directory where input files are located
mat_spect_files_path
    path to directory with .mat files containing spectrograms that should be used
    as data for which predictions are made
spect_scaler_path : str
    p